In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
#from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
dataset = pd.read_csv("kc_house_data.csv")

In [3]:
dataset.head()

,price,bedrooms,bathrooms,sqft_living,area,storeys,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
dataset.shape

(21613, 19)

In [5]:
# Identify features (X) and target variable (y)
X = dataset.drop('price', axis=1)
y = dataset['price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
X_test.head()

,bedrooms,bathrooms,sqft_living,area,storeys,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
735,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
2830,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
4106,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
16218,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
19964,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [7]:
y_test.head()

735       365000.0
2830      865000.0
4106     1040000.0
16218    1490000.0
19964     711000.0
Name: price, dtype: float64

In [47]:


# Initialize the CatBoostRegressor with hyperparameters
cb_model = CatBoostRegressor(
    iterations=200,  # Adjust the number of boosting iterations
    depth=7,         # Modify the depth of the tree
    learning_rate=0.1,  # Adjust the learning rate
    l2_leaf_reg=4.0,    # Try different values for regularization
    random_state=42 # You can set a random seed for reproducibility
)


In [48]:
# Train the model on the training data
cb_model.fit(X_train, y_train)

0:	learn: 338783.4720486	total: 12ms	remaining: 6s
1:	learn: 317971.8141344	total: 22.7ms	remaining: 5.65s
2:	learn: 300553.6177233	total: 33.4ms	remaining: 5.54s
3:	learn: 284022.7720861	total: 45.6ms	remaining: 5.66s
4:	learn: 269053.1516720	total: 56.3ms	remaining: 5.57s
5:	learn: 255701.2761330	total: 66.3ms	remaining: 5.46s
6:	learn: 244128.6138903	total: 75.7ms	remaining: 5.33s
7:	learn: 233750.2135764	total: 82.3ms	remaining: 5.06s
8:	learn: 224394.2595609	total: 90.5ms	remaining: 4.94s
9:	learn: 215373.7386160	total: 97.1ms	remaining: 4.76s
10:	learn: 207695.9138992	total: 106ms	remaining: 4.71s
11:	learn: 199502.2007822	total: 114ms	remaining: 4.63s
12:	learn: 193588.0555032	total: 122ms	remaining: 4.56s
13:	learn: 187606.3279680	total: 128ms	remaining: 4.44s
14:	learn: 181649.7271152	total: 143ms	remaining: 4.62s
15:	learn: 176557.2516792	total: 150ms	remaining: 4.52s
16:	learn: 172041.3773641	total: 158ms	remaining: 4.48s
17:	learn: 167938.5216597	total: 164ms	remaining: 4.4

In [49]:
# Make predictions on the test data
preds = cb_model.predict(X_test)


In [50]:
# Calculate Mean Squared Error (MSE) and R-squared (R2) Score
mse = mean_squared_error(y_test, preds)
cb_r2_score = r2_score(y_test, preds)

print("CatBoost R-squared (R2) Score:", cb_r2_score)
print("Mean Squared Error (MSE):", mse)

CatBoost R-squared (R2) Score: 0.9061480506735393
Mean Squared Error (MSE): 14204933937.925953


In [51]:
def predict_price(model, input_features):
    # Convert user input to a numpy array
    input_array = np.array(input_features).reshape(1, -1)

    # Make predictions
    predicted_price = model.predict(input_array)

    return predicted_price[0]

# Get user input
bedrooms = float(input("Enter the number of bedrooms: "))
bathrooms = float(input("Enter the number of bathrooms: "))
sqft_living = float(input("Enter the square footage of living space: "))
area = float(input("Enter the area: "))
storeys = float(input("Enter the number of storeys: "))
waterfront = float(input("Enter 1 if waterfront, 0 otherwise: "))
view = float(input("Enter the view rating: "))
condition = float(input("Enter the condition rating: "))
grade = float(input("Enter the grade: "))
sqft_above = float(input("Enter the square footage of roof: "))
sqft_basement = float(input("Enter the square footage of the basement: "))
yr_built = float(input("Enter the year built: "))
yr_renovated = float(input("Enter the year renovated: "))
zipcode = float(input("Enter the zipcode: "))
lat = float(input("Enter the latitude: "))
long = float(input("Enter the longitude: "))
sqft_living15 = float(input("Enter the square footage of living space for the nearest 15 neighbors: "))
sqft_lot15 = float(input("Enter the square footage of the lot for the nearest 15 neighbors: "))

# Create a list with user input features
user_input = [bedrooms, bathrooms, sqft_living, area, storeys, waterfront, view, condition, grade, sqft_above,
              sqft_basement, yr_built, yr_renovated, zipcode, lat, long, sqft_living15, sqft_lot15]


In [52]:
# Get the predicted price
predicted_price = predict_price(cb_model, user_input)

In [53]:
# Display the predicted price
print(f"The predicted price is: ${predicted_price:,.2f}")

The predicted price is: $253,568.69
